# Produccion electrica en base a la metereología

La idea principal del trabajo es ser capaces de predecir como va a afectar a que aumenten la produccion electrica de energias renovables las precipitaciones, el viento y las horas de sol.
Como las horas de sol y el viento son fenomenos cuya causa efecto es prácticamente inmediato vamos a centrarnos en la produccion de energia hidroelectrica.

En base a esto tenemos varias cuestiones que queremos despejar:
- Cuanto tarda en notarse en la generación eléctrica de fuentes de energía renovables los fenomenos metereológicos.
- Que relación hay entre precipitaciones y el aumento de la generación de energia eléctrica hidráulica.
- Existe realmente una relación entre la metereologia y la producción electrica.


Como premisas partimos de :
- Vamos a considerar solo el poll de energia que proporciona Red Electrica de España (REE)
- Vamos a considerar que las empresas no trabajan bajo mala praxis y que intentan optimizar el uso de energias renovables.
- Debido a la falta de datos a nivel diario de REE por provincia o comunidades autonomas, voy a centrar el analisis a nivel de sistema eléctrico (Peninsula, Baleares, Canarias ,Ceuta y Melilla).

Como origenes de datos para el estudio vamos a utilizar los datos proporcionados por:
- Información de REE (https://www.ree.es/es/apidatos) obtenida mediante su API.
- Datos proporcionados por aemet(https://opendata.aemet.es/centrodedescargas/inicio), vamos a utilizar la libreria  aemet desarrollada por Pablo Moreno (https://pypi.org/project/python-aemet/).

Requisitos para la ejecución del notebook:

Como requisitos para la ejecución del proyecto es necesario la instalación de la libreria python Aemet(pip install python-aemet) e instalar la libreria request



In [1]:
from aemet import Aemet,Estacion
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import json
import requests
import time

# Leemos la clave de la API de AEMET desde un fichero ubicado en la misma ruta que este notebook
with open('API_KEY_AEMET','r') as file:
    API_KEY_AE=file.read()

    

###  Lectura datos AEMET

Para la lectura de datos metereólogicos, vamos a utilizar la libreria aemet, de la que utilizaremos los metodos de las clases Aemet y Estacion para obtener los datos a nivel diario de cada estación meterologica para un rago de fechas, de las provincias que nos interesan para el estudio.

In [2]:
# Obtenemos el json de estaciones de mediciones de aemet 
info_estaciones=Estacion.get_estaciones(API_KEY_AE)

# Creamos un objeto Aemet para usar los metodos de la libreria aemet
aemet=Aemet(API_KEY_AE)

In [4]:

# Definimos funciones que vamos a utilizar para leer los datos de AEMET
def estaciones_prov (prov,lista_estaciones):
    '''Dada una lista de provincias y un json de estaciones de aemet. 
    Obtiene una lista de los ID de las estaciones de esa provincia.'''
    lista_id=[]
    prov=list(map(str.upper, prov))
    for estacion in lista_estaciones:
        if estacion['provincia'] in prov:
            lista_id.append(estacion['indicativo'])
    return lista_id


def lectura_diaria_json(date_ini,date_end,estaciones):
    '''Dado un json de la clase Estacion de la libreria Aemet, y fechas de inicio y fin:
    Obtenemos los datos climatologicos entre las dos fechas para todas las estaciones de manera diaria
    Si la fecha de inicio es anterior a 2016, se cambia a 2016-01-01, para evitar errores.
    '''
    valores_diarios=[]
    valores_error=[]
    if date_ini[0:4]<'2016-01':
        date_ini="2016-01-01T00:00:00UTC"
    if date_ini>date_end:
        print('Valores no válidos, fecha de inicio mayor que la fecha de fin')
        return valores_diarios;
    for element in estaciones:
        try:
            valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element['indicador'])
            if type(valores_estacion)!=dict:
                valores_diarios.extend(valores_estacion)
        except:
            time.sleep(56) # para evitar errores por nº de lecturas por minuto.
            #Volvemos a intentar leer el dato que ha lanzado la excepcion.
            try:
                valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element['indicador'])
                if type(valores_estacion)!=dict:
                    valores_diarios.extend(valores_estacion)
            except:
                print('Valor no encontrado')
                
    return valores_diarios;


def lectura_diaria_lista(date_ini,date_end,lista_estaciones):
    '''Dado una lista de id de estaciones de aemet, y fechas de inicio y fin:
    Obtenemos los datos climatologicos entre las dos fechas para todas las estaciones de manera diaria
    Si la fecha de inicio es anterior a 2016, se cambia a 2016-01-01, para evitar errores.
    '''
    valores_diarios=[]
    if date_ini[0:4]<'2016-01':
        date_ini="2016-01-01T00:00:00UTC"
    if date_ini>date_end:
        print('Valores no válidos, fecha de inicio mayor que la fecha de fin')
        return valores_diarios;
    for element in lista_estaciones:
        try:
            valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element)
            if type(valores_estacion)!=dict:
                valores_diarios.extend(valores_estacion)
        except:
            time.sleep(56) # para evitar errores por nº de lecturas.
            try:
                valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element)
                if type(valores_estacion)!=dict:
                    valores_diarios.extend(valores_estacion)
            except:
                print('Valor no encontrado')
                
    return valores_diarios;

In [ ]:
# Parametros para lectura de datos de AEMET

provincias=['Ceuta','Melilla','Illes Balears','Las Palmas','Sta. Cruz de Tenerife']

date_ini="2019-12-30T00:00:00UTC"
date_end="2019-12-30T23:59:59UTC"

id_estaciones=estaciones_prov(provincias,info_estaciones)

# Pasamos los datos ha dataframes para su procesado y limpieza
estaciones=pd.DataFrame(info_estaciones)
df_weather=pd.DataFrame(lectura_diaria_lista(date_ini,date_end,id_estaciones))

df_weather.drop(columns=['horaPresMax','horaPresMin','horaracha','dir','horatmin','horatmax'],inplace=True)

df_weather


In [13]:
# LIMPIEZA DE DATOS

df_weather.info()

# Idea: crear una funcion que para una columna dada, busque los NaN y aplique la media
tmax=mean
tmin=mean
tmed=mean
prec=mean
vlmedia=mean
sol=mean
pressMax=mean
pressMin=mean
racha-> mean
horaPresMax-> Eliminar
horaPresMin-> Eliminar
horaracha-> Eliminar
dir-> Eliminar
horatmin -> Eliminar
horatmax -> Eliminar

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
15,2019-12-30,C659M,"LAS PALMAS DE GRAN CANARIA, PL. DE LA FERIA",LAS PALMAS,15,NaN,"0,0",NaN,NaN,NaN,NaN,17,"1,7","6,4",12:10,NaN,"1023,5",Varias,"1020,1",02


In [17]:
df_wheater[df_wheater['provincia']=='LAS PALMAS']

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
12,2019-12-30,C249I,FUERTEVENTURA AEROPUERTO,LAS PALMAS,25,"18,2","0,0","14,7",07:50,"21,6",12:12,28,"4,2","8,3",08:34,"9,5","1022,9",Varias,"1020,0",05
13,2019-12-30,C649I,GRAN CANARIA AEROPUERTO,LAS PALMAS,32,"19,8","0,0","16,3",07:50,"23,4",13:30,35,"1,7","7,2",Varias,NaN,"1020,7",Varias,"1017,3",01
14,2019-12-30,C029O,LANZAROTE AEROPUERTO,LAS PALMAS,14,"19,4","0,0","14,5",23:23,"24,3",11:50,03,"4,2","8,3",05:10,"9,8","1023,9",Varias,"1021,1",05
15,2019-12-30,C659M,"LAS PALMAS DE GRAN CANARIA, PL. DE LA FERIA",LAS PALMAS,15,NaN,"0,0",NaN,NaN,NaN,NaN,17,"1,7","6,4",12:10,NaN,"1023,5",Varias,"1020,1",02
16,2019-12-30,C659H,"LAS PALMAS DE GRAN CANARIA, SAN CRISTOBAL",LAS PALMAS,55,"22,2","0,0","17,1",23:10,"27,3",14:40,18,"3,1","8,9",11:30,NaN,NaN,NaN,NaN,NaN
17,2019-12-30,C689E,MASPALOMAS,LAS PALMAS,6,"19,6","0,0","14,7",23:59,"24,4",14:30,06,"5,8","11,1",12:20,NaN,NaN,NaN,NaN,NaN
18,2019-12-30,C629X,"MOGÁN, PUERTO",LAS PALMAS,10,"22,2","0,0","17,7",04:10,"26,6",12:00,12,"5,0","12,8",11:20,NaN,NaN,NaN,NaN,NaN
19,2019-12-30,C229J,PÁJARA,LAS PALMAS,15,"21,1","0,0","18,0",23:30,"24,2",12:30,11,"4,7","20,6",13:50,NaN,"1025,4",11,"1022,1",02


# Análisis de la muestra

1 Evolución de temperaturas

2 Correlaciones

3 Evolución de precipitaciones

4 Datos estacionales

5 Localizar datos anomalos


### Lectura de los datos de REE

Para la lectura de los datos de REE voy a utilizar la libreria python requests para a traves de su API, obtener la generación de electricidad en , para cada tipo de energia Electrica.

Una vez leidos los datos de la API, guardo los datos en formato json en un fichero. para no tener que repetir las consultas y poder trabajar sin conexion.

La estrutura de los datos leidos de REE es la siguiente:


Por lo que vamos a almacenar 2 ficheros:
    - Renovables
    - No Renovables



In [18]:
# Leemos las regiones de ree obtenidas desde (https://www.ree.es/es/apidatos) desde un fichero ubicado en la misma ruta que este notebook
region_ree=pd.read_csv('REGION_REE',header=0,index_col='Region')

# Me quedo solo con los distintos sistemas electricos existentes
region_system=region_ree[region_ree['geo_limit']!='ccaa']

region_system

,geo_limit,geo_id
Region,,
peninsular,peninsular,8741
canarias,canarias,8742
baleares,baleares,8743
ceuta,ceuta,8744
melilla,melilla,8745


In [165]:
geo_limit=region_system[region_system['geo_id']==8745]['geo_limit']
geo_limit[0]

'melilla'

In [ ]:
def lectura_ree(d_inicio,d_fin,geo_id):
    
    # meter esto en una funcion con su try-exception    
    
    geo_limit=region_system[region_system['geo_id']==geo_id]['geo_limit']
    
    parametros={'start_date':'2020-12-31T00:00',
            'end_date':'2021-01-20T00:00',
            'time_trunc':'day',
            'geo_trunc':'electric_system',
            'geo_limit':geo_limit[0],
            'geo_ids':geo_id}
    
    URL_GEN='https://apidatos.ree.es/es/datos/generacion/estructura-generacion'

    ree_gen=requests.get(URL_GEN,params=parametros)
    
    return ree_gen
    

In [152]:
# Obtenemos los datos de REE a traves de su API.




parametros={'start_date':'2020-12-31T00:00',
            'end_date':'2021-01-20T00:00',
            'time_trunc':'day',
            'geo_trunc':'electric_system',
            'geo_limit':'canarias',
            'geo_ids':'8742'}

URL_GEN='https://apidatos.ree.es/es/datos/generacion/estructura-generacion'

ree_gen=requests.get(URL_GEN,params=parametros)
  
if ree_gen.raise_for_status()!=None:
    print('Error en lectura')


In [157]:
df_ree=pd.DataFrame()
for i in range(10):
    df=pd.json_normalize(ree_gen.json()['included'][i]['attributes'],meta=['title','type'],record_path=['values'])
    df_ree=df_ree.append(df)
    
df_ree.reset_index(inplace=True,drop=True)
df_ree

,value,percentage,datetime,title,type
0,9.902,0.000446,2020-12-31T00:00:00.000+01:00,Hidráulica,Renovable
1,5443.054,0.245243,2020-12-31T00:00:00.000+01:00,Motores diésel,No-Renovable
2,4260.363,0.216781,2021-01-01T00:00:00.000+01:00,Motores diésel,No-Renovable
3,5212.173,0.244999,2021-01-02T00:00:00.000+01:00,Motores diésel,No-Renovable
4,5229.706,0.253130,2021-01-03T00:00:00.000+01:00,Motores diésel,No-Renovable
...,...,...,...,...,...
163,20931.266,1.000000,2021-01-16T00:00:00.000+01:00,Generación total,Generación total
164,19821.599,1.000000,2021-01-17T00:00:00.000+01:00,Generación total,Generación total
165,21840.651,1.000000,2021-01-18T00:00:00.000+01:00,Generación total,Generación total
166,21879.485,1.000000,2021-01-19T00:00:00.000+01:00,Generación total,Generación total
